## Лекция 7. Градиентный бустинг

Кратко вспомним материалы прошлой лекции.

### Bias and Variance

Модели машинного обучения подвержены ошибкам трех видов:

* Неустранимая ошибка целевой переменной
* Ошибка смещения (**Bias**) - "недообучение"
* Ошибка разброса (**Variance**) - "переобучение"

Искусство построения моделей - поиск оптимальной точки между смещением и разбросом (**Bias-Variance trade-off**)

<img src="img/0_bias_variance_0.jpg" height=80% width=80%>
<img src="img/0_bias_variance_1.jpg" height=80% width=80%>

### Ансамбли моделей

* Объединение моделей в ансамбли позволяет повысить качество
* Улучшение происходит за счет сглаживания индивидуальных предсказаний моделей
* Если $p$ (вероятность правильного ответа) > 0.5, то у нас все должно сложиться хорошо
* Чем разнообразнее модели в ансамбле, тем лучше

Пусть на выборке $(X, y)$ обучены базовые модели - $b_1(x)$, $b_2(x)$, ... $b_m(X)$ 

Тогда (в задаче классификации) для "учета мнения" всех базовых алгоритмов используют:

* Простое голосование большинством

$$ f(x) = \dfrac{1}{m}\sum^{m}_{i=1}b_i(x) $$

* Взвешенное голосование

$$ f(x) = \dfrac{1}{m}\sum^{m}_{i=1}w_ib_i(x) $$

Где $w_i$ - вес соответствующей модели $b_i$ в ансамбле

Та же самая логика работает и в случае задачи регрессии

### Bagging & Random Forest

Было рассмотрено 3 подхода к построению ансамблей из независимых моделей:

* Bagging
* Метод случайных подпространств
* Random Forest

<img src="img/1_tree_ensembles.jpg" height=80% width=80%>

* В алгоритме Random Forest используются **решающие деревья**
* Каждое дерево **независимо** обучается на случайной подвыборке объектов со случайно выбранными признаками

<img src="img/1_tree_picture.jpg" width=80% height=80%>

<img src="img/1_rf_prediction.jpg" height=80% width=80%>

* Почему решающее дерево - хороший кандидат для базовой модели в бэггинге/случайном лесе?
* Зачем в случайном лесе брать подпространство признаков?

### Интуиция: глобально

* Строили базовые модели независимо друг от друга
* А что если вместо этого строить модели последовательно, на каждой итерации учитывая "опыт прошлых поколений"?
* Назовем такой подход бустингом (**boosting**)

Итоговый ансамбль:
$$\large a(x)=\sum^{T}_{i=1}w_ib_i(x)$$

Основное отличие от Random Forest - здесь модель $b_i(x)$ была построена на i-ой итерации алгоритма.

### Интуиция: weights-based boosting

Начнем с подхода под названием _**"weights-based boosting"**_ - бустинг, основанный на весах объектов.

<img src="img/1_wbb_0.jpg">

$$\large a(x)=b_1(x)$$

<img src="img/1_wbb_1.jpg">

Обучили базовый алгоритм $b_1(x)$, применили его на обучающей выборке и получили предсказанные вероятности $\large y_{pred,1}$

$$\large a(x)=b_1(x)$$

<img src="img/1_wbb_2.jpg">

После чего перевзвесим объекты:

$$\large new\_weight_i = \dfrac{abs\_error_i}{\sum_jabs\_error_j}$$

<img src="img/1_wbb_3.jpg">

После первой итерации имеем:

* Веса теперь не распределены равномерно
* Модель $b_2$ будет уделять "больше внимания" объектам, которые после предсказания от $b_1$ получили больший вес.

$$\large a(x) = b_1(x) + b_2(x)$$

Проделаем аналогичные шаги для обновленного веса:

<img src="img/1_wbb_4.jpg">
<img src="img/1_wbb_5.jpg">

Будем снова перевзвешивать объекты, однако теперь мы будем использовать ошибку модели $\large b_2(x)$
<img src="img/1_wbb_6.jpg">
<img src="img/1_wbb_7.jpg">

Шаг 3. Повторяем описанную выше процедуру

$$\large a(x) = b_1(x) + b_2(x) + b_3(x) $$

...

### Интуитивные соображения:

* Каждая новая модель будет пытаться "исправить" ошибки предыдущей
* Используем _learning rate_ ("темп обучения"), чтобы не получалось так, что предсказания некоторых моделей не "задавливали" предсказания других

$$\large a(x) = \eta b_1(x) + \eta b_2(x) + \eta b_3(x)$$

### Бустинг - задача бинарной классификации

Обучающая выборка $(X, y)$ из $N$ объектов, $y \in \{-1, 1\}$

Модель взвешенного голосования:

$$\large a(x)=sign(\sum^{T}_{i=1}w_i b_i(x)) $$

Функция потерь:

$$\large loss = \dfrac{1}{N}\sum^{N}_{j=1}I(y_j \neq a(x_j))$$

<img src="img/2_losses.jpg" height=80% width=80%>

* Missclassification $= I(sign(f) \neq y)$
* Exponential $= exp(-yf)$

Оптимизация _exponential loss_ приводит нас к алгоритму **AdaBoost**

### AdaBoost

**Алгоритм:**

* Задаем начальное распределение весов объектов в обучающей выборке:
    $$\large\alpha_j^0 = \dfrac{1}{N}, j=1,2,\dots,N$$

* Для каждого ***i*** от ***1*** до ***T***:
  * Строим классификатор $b_i$ с использованием весов $\{w^{i-1}\}$
  * Вычисляем ошибку $b_i$: $$\large err_i = \sum^{N}_{j=1}\alpha^{i-1}_j I[y_j \neq b_i(x_j)]$$
  * Вычисляем весовой коэффициент для $b_i$: $$\large w_i = \dfrac{1}{2}ln\dfrac{1 - err_i}{err_i}$$
  * Обновляем веса объектов: $$\large \alpha_j^i = \alpha_j^{i-1}e^{w_iI[y_j \neq b_i(x_j)]}$$
  * Нормируем веса обучающих объектов: $$\large \alpha_j^i = \dfrac{\alpha_j^i}{\sum_{k=1}^{N}\alpha_k^i}$$

### Пример на пеньках

* Решаем задачу классификации
* Базовые алгоритмы - "пни" (решающие деревья глубины 1)
* 10 объектов в выборке
* Их начальные веса - 0.1

<img src="img/4_pen_0.jpg" height=30% width=30%>

* Строим пень #1
* Он ошибается на 3 объектах
* Его вес - 0.42

<img src="img/4_pen_1.jpg" height=30% width=30%>

* Увеличиваем вес объектов с ошибкой

<img src="img/4_pen_2.jpg" height=30% width=30%>

* Строим пень #2
* Также ошибается на 3 объектах
* Однако его вес уже 0.65, т.к. он лучше определяет более "тяжелые" объекты

<img src="img/4_pen_5.jpg" height=30% width=30%>

* Увеличиваем вес объектов с ошибкой

<img src="img/4_pen_6.jpg" height=30% width=30%>

* Строим пень #3
* Снова ошибается на 3 объектах
* Вес этого пня - 0.92

<img src="img/4_pen_7.jpg" height=30% width=30%>

Итоговый алгоритм - взвешенная композиция базовых алгоритмов

<img src="img/4_pen_9.jpg" height=60% width=60%>

### Экспериментальный факт

<img src="img/5_fact.jpg">

### Residual-based boosting

_Residual-based boosting_ = бустинг, основанный на остатках

$$\large a(x) = b_1(x)$$

<img src="img/6_resid_0.jpg">

* Строим модель $b_1(x)$
* Делаем предсказания при помощи этой модели

<img src="img/6_resid_1.jpg">

$$\large a(x) = b_1(x)$$

<img src="img/6_resid_2.jpg">

Для каждого объекта из обучающей выборки считаем невязку (**residual**) модели $b_1(x)$: $y - y_{pred}$

<img src="img/6_resid_3.jpg">

Делаем полученную невязку таргетом для обучения следующей модели - $b_2(x)$

$$\large a(x) = b_1(x)$$

<img src="img/6_resid_4.jpg">

$$\large a(x) = b_1(x) + b_2(x)$$

<img src="img/6_resid_5.jpg">

Снова считаем невязку

<img src="img/6_resid_7.jpg">

$$\large a(x) = b_1(x) + b_2(x)$$

<img src="img/6_resid_8.jpg">

И делаем таргетом для модели $b_3(x)$

Проделываем аналогичные шаги для $b_3(x)$, $b_4(x)$, $b_5(x)$, ..., $b_T(x)$

Итоговый ансамбль выглядит так:

$$\large a(x) = b_1(x) + b_2(x) + b_3(x) + \dots + b_T(x) = \sum^T_{i=1}b_i(x)$$

### Gradient Boosting

* Модель: $$\large a(x) = \sum^{T}_{i=1}b_i(x)$$

* Ошибка композиции на train выборке:

$$\large err(a) = \sum^{N}_{j=1}L(y_j, a(x_j) = \sum^{N}_{j=1}L(y_j, \sum^{T}_{i=1}w_ib_i(x)) = $$
$$\large = \sum^{N}_{j=1}L(y_j, \sum^{T-1}_{i=1}w_ib_i(x) + wb(x_j) \rightarrow \underset{w,b}{min}$$

* На i-ой итерации при построении модели все предыдущие зафиксированы

### Gradient Boosting: алгоритм

* Инициализируем ансамбль:

$$\large a_0(x) = \underset{C}{argmin}\sum^{N}_{i=1}L(y_i, C)$$

* Для всех i от 1 до **T**:
  * Для всех $j$ от $1$ до $N$ вычисляем градиент функции потерь по ответам ансамбля $\large g_{ij}$:
    $$\large g_{ij} = -[\dfrac{\partial L(y_i, a(x_j)}{\partial a(x_j)}]_{a=a_{i-1}}$$
    
  * Строим $\large b_i$ с целевой переменной $\large g_{ij}$:
    $$\large b_i = \underset{b}{argmin}\space(\sum^N_{j=1}(g_{ij} - b(x_j))^2$$
    
  * Вес $w_i$ модели $b_i$:
    $$\large w_i = \underset{w}{argmin}\space\sum^N_{j=1}L(y_j, a_{i-1}(x_j) + wb_i(x_j))$$
    
  * Добавляем в ансабмль:
    $$\large a_i(x) = a_{i-1}(x) + w_ib_i(x)$$
* Ансамбль на выходе: $$\large a(x) = a_T(x)$$

### Gradient Boosting: задача регрессии

* Задача регрессии, функция потерь - MSE
* Модель - **a(x)**
* Функция потерь на одном объекте $(x_j, y_j)$:

$$\large L_j = L(y_j, a(x_j) = MSE(y_j, a(x_j)) = \dfrac{1}{2}(y_j - a(x_j))^2$$

* Анти-градиент функции потерь по модели **a**:

$$\large -\dfrac{\partial L_j}{\partial a} = - \dfrac{1}{2} \dfrac{\partial(y_j - a(x_j))^2}{\partial a} = 
  y_j - a(x_j)$$
  
* MSE на одном объекте - разница между реальным значением и таргетом (как в интуитивном примере выше)

### Gradient Boosting: learning rate

* Полученные на i-ой итерации модели с какой-то точностью аппроксимируют градиент функции потерь
* Не стоит полностью доверять данной аппроксимации
* Поэтому добавим постоянный коэффициент $\eta$ перед моделями - **learning_rate**:

$$\large a_i(x) = a_{i-1}(x) + \eta w_ib_i(x)$$

### Stochastic Gradient Boosting

* Использование bagging'a при построении деревьев может улучшить обощающую способность
* Больше вероятность попадания в локальный минимум функции потерь
* Можно использовать OOB error

<img src="img/5_shrinkage.jpg" height=60% width=60%>

### XGBoost

* Библиотека для обучения моделей градиентного бустинга
* ~ Extreme Gradient Boosting
* Особое внимание уделено регуляризации деревьев
* Разрабатывается с 2014 года

https://github.com/dmlc/xgboost

### XGBoost: регуляризация деревьев

Основная идея - модификация критерия разбиения

$$\large Impurity_{node} = -\dfrac{1}{2}\dfrac{G_j^2}{H_j + \lambda} + \gamma T$$

https://xgboost.readthedocs.io/en/latest/tutorials/model.html

### LightGBM

Библиотека для градиентного бустинга от Microsoft

Быстрее XGBoost

Деревья строятся в глубину (**Leaf-wise tree growth**)

https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf

### Обзор настроек

* Общие параметры:
  * Тип задачи (objective)
  * Функция потерь (loss) - оптимизируемая величина
  * Метрика качества (eval_metric) - контрольная величина
* Параметры композиции
  * Темп обучения (learning_rate)
  * Число итераций (n_estimators)
* Параметры одного дерева
  * Максимальная глубина (max_depth)
*
...
* Параметры сэмплирования
  * Доля объектов / признаков для построения дерева


### Итоги:

* На больших и сложных объемах данных - один из лучших алгоритмов
* Много гиперпараметров
* Обычно строится на деревьях решений
* Есть быстрые реализации